<a href="https://colab.research.google.com/github/Nagamani732/DeepLearning.ai-Summary/blob/master/TF_V1_x_x_Inference_using_frozen_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.15.3

In [2]:
import tensorflow as tf # Default graph is initialized when the library is imported
from tensorflow.python.platform import gfile
import numpy as np
import cv2
import os
import sys

In [12]:
BATCH = 1
HEIGHT = 224
WIDTH = 224
CHANNEL = 3
MODEL_FILE = "mobilenet_v2_1.0_224.pb"
INPUT_IMAGE = "input_image.jpg"

In [13]:
def FetchInputOutput(sess):
    inputs = []
    for op in sess.graph.get_operations():
        if (op.type == "Placeholder"):
            inputs.append(op.name.replace('import/', ''))
    if (len(inputs) == 0): 
        print("No inputs spotted.")
    else:
        print("Found ", len(inputs), " possible input(s): ")
        for i in range(len(inputs)):
                print(inputs[i])
    node_inputs = []
    outputs = []
    for node in sess.graph_def.node:
        for input in node.input:
            node_inputs.append(input)
    unlikely_output_types = ["Const", "Assign", "NoOp", "Placeholder"]
    for op in sess.graph.get_operations():
        if ((node_inputs.count(op.name) == 0) and (unlikely_output_types.count(op.type) == 0)):
            outputs.append(op.name.replace('import/', ''))
    if (len(outputs) == 0): 
        print("No outputs spotted.")
    else:
        print("Found ", len(outputs), " possible output(s): ")
        for i in range(len(outputs)):
                print(outputs[i])
    return inputs, outputs

In [39]:
with tf.Graph().as_default() as graph:
    with tf.Session() as sess:
        # Load the graph in graph_def
        print("\nLoad graph")
        if not os.path.isfile(MODEL_FILE):
            print('File does not exist')
            sys.exit()
        if not os.path.isfile(INPUT_IMAGE):
            print('Input image does not exist') 
            sys.exit()
        # Load the protobuf file from the disk and parse it to retrieve the unserialized graph_def
        with gfile.FastGFile(MODEL_FILE,'rb') as f:
            # Set graph to the default graph
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            # Import a graph_def into the current default Graph (In this case, the weights 
            # are (typically) embedded in the graph)
            tf.import_graph_def(graph_def, name="")
            inputs, outputs = FetchInputOutput(sess)
            input_names = []
            output_names = []
            for input in inputs:
                input_names.append(input+':0')
            for output in outputs:
                output_names.append(output+':0')  
            # Inference 
            _input = graph.get_tensor_by_name(input_names[0]) # Input Tensor
            _output = graph.get_tensor_by_name(output_names[0]) # Output Tensor
            print("\nLoad Image...")
            # Read the image
            oriimg = cv2.imread(INPUT_IMAGE, cv2.IMREAD_COLOR)
            oriimg  = oriimg.astype(float)/255
            resize_image = cv2.resize(oriimg,(WIDTH, HEIGHT))
            net_image = np.reshape(resize_image, [BATCH, HEIGHT, WIDTH, CHANNEL])
            #initialize_all_variables
            tf.global_variables_initializer()
            # Run model on single image
            print("\nRun model...")
            out = sess.run(_output, feed_dict= {_input:net_image})
            top5 = tf.nn.top_k(out, 5)
            print("Top5 Probablities...\n")
            ind = sess.run(top5).indices
            val = sess.run(top5).values
            for i in range(0, 5):
              if os.path.isfile('labels.txt'):
                with open('labels.txt') as f:
                  for line in f:
                    index = ind[0][i]
                    if str(index-1)+':' in line:
                      print(line.strip() + "  " + str(val[0][i]))
                      break
              else:
                print(ind[0][i], " ", val[0][i])


Load graph
Found  1  possible input(s): 
Placeholder
Found  1  possible output(s): 
MobilenetV2/Predictions/Softmax

Load Image...

Run model...
Top5 Probablities...

208: 'Labrador retriever',  0.45184022
222: 'kuvasz',  0.119468376
170: 'Irish wolfhound',  0.033233084
181: 'Bedlington terrier',  0.02925248
182: 'Border terrier',  0.024229426
